# Rollers Pitch-Shift Implmenetation

The "Rollers" pitch-shifting algorithm is based on narrow subband frequency shifting.
For yielding low latencies, an IIR filter bank is used.

The original IIR filter bank implementation in Java uses Butterworth bandpass filters with crossovers at -12dB.

Things to figure out:

* scipy filter design

* How to design a Butterworth filter bank

* Frequency shifting

* Putting it all together

Note: Keras GPU Filter Bank


## Scipy filter design

analog filter design:

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 3]

# filter coeffs
b, a = signal.butter(4, 100, 'low', analog=True)
print('b:', b, 'a:', a)
# frequency response
w, h = signal.freqs(b, a)

plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth analog filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

digital fiter design:

Default output format is ‘ba’ for backwards compatibility, but ‘sos’ should be used for general-purpose filtering.

In [ ]:
fs = 44100
fc = 100
# normalized cutoff frequency
wc = fc / (fs / 2)
b, a = signal.butter(4, wc, 'low', analog=False)
print('b:', b, 'a:', a)

w, h = signal.freqz(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth digigtal filter frequency response')
plt.xlabel('normalized Frequency (pi is nyquist freq))')
plt.ylabel('Amplitude [dB]')
plt.axvline(wc*np.pi, color='green') # cutoff frequency
plt.show()

Now we use the recommended second-order sections format when filtering, to avoid numerical error with transfer function (ba) format.

In [ ]:
b, a = signal.butter(4, fc, 'low', fs=fs, output='ba')
sos = signal.butter(4, fc, 'low', fs=fs, output='sos')

w, h = signal.freqz(b, a)
plt.semilogx(w, 20*np.log10(np.abs(h)))
plt.show()

## Filtering

In [ ]:
x = signal.unit_impulse(1024)

y_tf  = signal.lfilter(b, a, x) # ba format
y_sos = signal.sosfilt(sos, x)  # sos format

plt.plot(y_tf, 'r', label='TF')
plt.plot(y_sos, '--k', label='SOS')
plt.legend(loc='best')
plt.show()

In [ ]:
from scipy.fft import rfft, rfftfreq
freq = rfftfreq(x.size, 1 / fs)
plt.semilogx(freq, 20*np.log10(np.abs(rfft(y_sos))))
plt.show()

### How to design a band pass filter

In [ ]:
def butter_bp(lowcut, highcut, fs, order=4, t='sos'):
    f_nyq = 0.5 * fs
    low = lowcut / f_nyq
    high = highcut / f_nyq
    return signal.butter(order, [low, high], btype='band', output=t)

In [ ]:
lowcut = 500
highcut = 1000

b, a = butter_bp(lowcut, highcut, fs, order=5, t='ba')

# plot
w, h = signal.freqz(b, a)
plt.semilogx((fs * 0.5 / np.pi) * w[1:], 20*np.log10(np.abs(h[1:])))
plt.ylim((-40, 5))
plt.show()

## Filter Bank Design

Let's design a constant Q Butterworth bandpass filter bank.
There are different possible center frequency spacings:

* [Third-Octave Filter Banks](https://ccrma.stanford.edu/realsimple/aud_fb/Third_Octave_Filter_Banks.html)

* [ERB Filter Bank](https://ccrma.stanford.edu/realsimple/aud_fb/Equivalent_Rectangular_Bandwidth_ERB.html)

* [Mel Scale](https://labrosa.ee.columbia.edu/doc/HTKBook21/node54.html)

* Bark Scale

Let's start with a **third-octave filter bank**:

In [ ]:
fs = 44100
n = 28

# third-octave filter bank
freq_offset = 2
k = np.arange(n+2) - n // 2 - freq_offset

# center frequencies are defined relative to a bandpass with center frequency at 1kHz
f_cs = np.power(2, k / 3) * 1000
print('f_cs:', f_cs)

f_chs = [] # high cutoff frequencies
f_cls = [] # low cutoff frequencies
filters = []
for k in range(1, f_cs.size-1):
    f_chs.append(np.sqrt(f_cs[k] * f_cs[k+1]))
    f_cls.append(np.sqrt(f_cs[k-1] * f_cs[k]))
    
for k in range(f_cs.size-2):
    sos = butter_bp(f_cls[k], f_chs[k], fs, order=4, t='sos')
    filters.append(sos)
    
# plot
for sos in filters:
    w, h = signal.sosfreqz(sos, worN=10000)
    plt.semilogx((fs * 0.5 / np.pi) * w[1:], 20*np.log10(np.abs(h[1:])))
    plt.ylim((-100, 5))
    plt.xlim((10, 20000))
    plt.ylabel('H [dB]')
    plt.xlabel('f [Hz]')
    plt.title('third-octave filter bank')
plt.show()